<a href="https://colab.research.google.com/github/aperop/ds/blob/main/day06/ex04/ex04_ab-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [36]:
file_path = '../data/checking-logs.sqlite'
file_path = 'data/checking-logs.sqlite'

In [37]:
con = sqlite3.connect(file_path)
cur = con.cursor()

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

In [38]:


query = 'PRAGMA table_info(test);'
pd.read_sql(query, con=con)



,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [39]:
query = '''
with test_users as (
select
    uid
from
    (select distinct
        uid,
        case
            when first_commit_ts > first_view_ts then
                'after'
            else 'before'
        end time
    from test)
group by uid
having count(time) >= 2)
select
    case
        when first_commit_ts > first_view_ts then
            'after'
        else 'before'
    end time,
    avg(cast((-JulianDay(test.first_commit_ts) + (JulianDay(test.first_view_ts))) * 24 as Integer)) avg_diff
from test
where uid in test_users
group by time

'''

test_result = pd.read_sql(query, con)

In [40]:
test_result

,time,avg_diff
0,after,-368.800000
1,before,155.421053


In [41]:
query = '''
with control_users as (
select
    uid
from
    (select distinct
        uid,
        case
            when first_commit_ts > first_view_ts then
                'after'
            else 'before'
        end time
    from control)
group by uid
having count(time) >= 2)
select
    case
        when first_commit_ts > first_view_ts then
            'after'
        else 'before'
    end time,
    avg(cast((-JulianDay(first_commit_ts) + (JulianDay(first_view_ts))) * 24 as Integer)) avg_diff
from control
where uid in control_users
group by time

'''
control_result = pd.read_sql(query, con)

In [42]:
control_result

,time,avg_diff
0,after,-329.630435
1,before,104.689655


## close the connection

In [43]:
con.close()

## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

По этому A/B нельзя однозначно определить верна ли гипотеза.